In [1]:
from dotenv import load_dotenv
from lite_agent.utils.litellm_utils import inspect_litellm_request

load_dotenv()

True

In [2]:
import json
# d = inspect_litellm_request(
#         model="anthropic/claude-3-5-sonnet-20240620",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": "Explain quantum computing."}
#         ],
#         max_tokens=2048
#     )
d = inspect_litellm_request(
        model="anthropic/claude-3-5-sonnet-20240620",
        messages=[
            {"role": "user", "content": "What's the weather in San Francisco?"}
        ],
        temperature=0.7,
        max_tokens=1024,
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "description": "Get weather information for a location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        ],
        tool_choice="auto"
    )
# d = inspect_litellm_request(
#         model="anthropic/claude-3-5-sonnet-20240620",
#         messages=[
#             {"role": "user", "content": "Say hello there!"}
#         ],
#         temperature=0.7,
#         max_tokens=1024
#     )
print(json.dumps(d['transformed_params'], indent=2, default=str))

{
  "model": "claude-3-5-sonnet-20240620",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What's the weather in San Francisco?"
        }
      ]
    }
  ],
  "temperature": 0.7,
  "max_tokens": 1024,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "get_weather",
        "description": "Get weather information for a location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
            }
          },
          "required": [
            "location"
          ]
        }
      }
    }
  ],
  "tool_choice": "auto",
  "stream": false
}


In [7]:
# Test your exact configuration
result = inspect_litellm_request(
    model="anthropic/claude-sonnet-4-5",
    messages=[{"role": "user", "content": "Your message here"}],
    max_tokens=64000,
    thinking={"type": "enabled", "budget_tokens": 12400},
    tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
    betas=["files-api-2025-04-14", "code-execution-2025-08-25"]
)

print(json.dumps(result['transformed_params'], indent=2, default=str))

{
  "model": "claude-sonnet-4-5",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Your message here"
        }
      ]
    }
  ],
  "max_tokens": 64000,
  "tools": [
    {
      "type": "code_execution_20250825",
      "name": "code_execution"
    }
  ],
  "stream": false,
  "thinking": {
    "type": "enabled",
    "budget_tokens": 12400
  },
  "betas": [
    "files-api-2025-04-14",
    "code-execution-2025-08-25"
  ]
}


In [4]:
import anthropic

# Initialize the client
client = anthropic.Anthropic()

## Anthropic - Files API

In [ ]:


# Step 1: Upload a file using the Files API
with open('files/tata-motor-IAR-2024-25.pdf', 'rb') as file:
    uploaded_file = client.beta.files.upload(
        file=('tata-motor-IAR-2024-25.pdf', file, 'application/pdf')
    )

file_id = uploaded_file.id
uploaded_file

FileMetadata(id='file_011CV2R3aJGA4a655cY3DEYP', created_at=datetime.datetime(2025, 11, 11, 16, 26, 43, 271000, tzinfo=datetime.timezone.utc), filename='tata-motor-IAR-2024-25.pdf', mime_type='application/pdf', size_bytes=50707487, type='file', downloadable=False)

## Anthropic Call

In [18]:
# Stream text and collect the final message
with client.messages.stream(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Say hello there!",
        }
    ],
    model="claude-sonnet-4-5",
) as stream:
    # Stream the text output
    for text in stream.text_stream:
        print(text, end="", flush=True)
    print()

# Get the accumulated final message (available after context manager exits)
accumulated = stream.get_final_message()


Hello there! 👋 

How can I help you today?


In [6]:
print("accumulated message: ", accumulated.to_json())

accumulated message:  {
  "id": "msg_01NDJmfREj7DkU5MxJytJoYK",
  "content": [
    {
      "citations": null,
      "text": "Hello there! 👋 \n\nHow can I help you today?",
      "type": "text"
    }
  ],
  "model": "claude-sonnet-4-5-20250929",
  "role": "assistant",
  "stop_reason": "end_turn",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "cache_creation": {
      "ephemeral_1h_input_tokens": 0,
      "ephemeral_5m_input_tokens": 0
    },
    "cache_creation_input_tokens": 0,
    "cache_read_input_tokens": 0,
    "input_tokens": 11,
    "output_tokens": 18,
    "service_tier": "standard"
  }
}


In [16]:
accumulated.model_dump(include={"role", "content"})

{'content': [{'citations': None,
   'text': 'Hello there! 👋 \n\nHow can I help you today?',
   'type': 'text'}],
 'role': 'assistant'}

In [21]:
messages = [
    {
        "role": "user",
        "content": "Say hello there!",
    }
]

messages.append(
    accumulated.model_dump(include={"role", "content"})
)
# messages.append(
#     {"role": accumulated.role, "content": accumulated.content}
# )
messages.append(
    {
        "role": "user",
        "content": "What did you do in previous step?",
    }
)
with client.beta.messages.stream(
    model="claude-sonnet-4-5",
    max_tokens=2048,
    thinking={
        "type": "enabled",
        "budget_tokens": 1024,
    },
    messages=messages,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)
    print()

accumulated = stream.get_final_message()
print("accumulated message: ", accumulated.to_json())


I apologize for the confusion in my last response! I made an error.

Looking back at the conversation:
- In the previous step, YOU said "Say hello there!" 
- I responded to that greeting

Then you asked "What did you do in previous step?" and I incorrectly described what had happened. 

To be clear: In my first response to you, I said hello back to you. That was the previous step before your current question.
accumulated message:  {
  "id": "msg_019Je8DsxHAntDMhxoQFdBuq",
  "container": null,
  "content": [
    {
      "signature": "Ev8HCkYIChgCKkDhSaNfap1SZ5gE7FSVlrc194WrEwO76XuAB3Dt5PkK/3ZbbS/nw8RYw6PXRx6Ew+2gWXLB3cNfhJ85oFSClOb5EgxVnvfWihzNuBb1B1gaDEw80ZfjCOCWT1MCWyIwQaCwSG8NjqRnDsCaPd7Gbry619RtjjSdLMn5jeaXg/7tz4zQPBZIKVqKDzMh8n5+KuYGjzFUstqq44XGunztXpBbccWxbOQ/XB64ZFX7UNXogZUhodrUL2zTcef7TWf9gTI6/542zu/m2KxT/xd3d4oUk3uUIkwxTyneba1hWYpq/LMf7mqOKyamglfwO6utUOLl+gVCdFaRJ9ux4eYbWPPi3icRYvycdtEx6cbsfa3O3JX6AuhoJCNJPSTR95QSKS/w1hNHSXHz3dvRhRBjimNES5mxgdYDD0/0AdyDDrCjbc96e24MifofSvWXuhiq0

In [ ]:
# Step 2: Use the file with streaming and thinking enabled
# Build the message content
message_content = [
    {"type": "container_upload", "file_id": file_id},
    {"type": "text", "text": "Extract the tata motors balance sheet from this annual report and place it in an xlsx file."}
]

with client.beta.messages.stream(
    model="claude-sonnet-4-5",
    thinking={
        "type": "enabled",
        "budget_tokens": 12400
    },
    max_tokens=64000,
    betas=["files-api-2025-04-14", "code-execution-2025-08-25"],
    messages=[
        {
            "role": "user",
            "content": message_content
        }
    ],
    tools=[{
        "type": "code_execution_20250825",
        "name": "code_execution"
    }]
) as stream:
    # Stream the text output
    for text in stream.text_stream:
        print(text, end="", flush=True)
    print()

# Get the accumulated final message (available after context manager exits)
accumulated = stream.get_final_message()

I'll help you extract the balance sheet from the Tata Motors annual report and create an Excel file. Let me start by examining the PDF file.Now let me create a Python script to extract the balance sheet from the PDF:Let me try with a different PDF library:Let me continue to see the actual output:Now let me extract the actual balance sheet content from page 167:This doesn't seem to be the main balance sheet. Let me check page 257 and nearby pages more systematically:Good! I found the balance sheet summary on page 257. Now let me look for the detailed balance sheet which is typically in the financial statements section. Let me check pages around 298-304:Let me now look at the actual balance sheet which should be in the consolidated financial statements:Perfect! I found the Consolidated Balance Sheet on page 312. Now let me extract the complete balance sheet with all items:Excellent! Now I have the complete consolidated balance sheet. Let me create a comprehensive Excel file with this dat

In [10]:
print("accumulated message: ", accumulated.to_json())

accumulated message:  {
  "id": "msg_01S7uKPuTyYaEdyb6cFPHZZ5",
  "container": {
    "id": "container_011CV2TTXsJmbXMw6hXuX3ms",
    "expires_at": "2025-11-11T18:03:27.848887Z"
  },
  "content": [
    {
      "signature": "EoQFCkYICRgCKkBviY6exQY0N0be1/EGZm4sbtYeVHpMJFTUjRfexTRgVmear8rU2dEoBwd7Nuxb0ATJjtHmBGRk6vqoZNBuZbuNEgyKrBE5rG+J03IFzXkaDD2C8C8FF80PQAn1cCIwhWDnXByZwnbQlk2US4DGuswEmOrKJyROI9MM9xggWUV+V7RXpEItB/TbRn8jb7abKusD8LTnabMB0xDkRIBtwOfdRQyxlg6GySx59s3Di4Pk0xalawN7uWcf43QUIkdW2LfdBvZ4r/7rVSoNpm/cmh/UI5SZmk+GHE8szGKSAKLdoFos3wH9tQUJcpkBsuYUpMlt73ykw/0M1aSqV2h1PFb7xoXFP5RQx6Re+0NHj7vo86tkyXV1oqL0m5VKSv4/vubo+Jgq+HdLPJUNsEvtnLXcivsm1pyK5E2sOtHTvnHms7baao3Og1O9u4RNl8iaQUgPofCKw/w34CjvqHNhP/UCwZ31VCb+IkIRr03tC2cRpqNCBxUQCR89Xn3eN17/PfaTgdoAQPrK582G67YxgeoPhWF6rVWpyH9N8gC5BSW0rRhlNtgSAgZiO7rbkSfhPQN2ZkS3wOE73hP6xFNzVGDYV0kZ5GjyATX2T50LPkDq+D6naS8UUAZx+35DpTXRR7wyNGFrg2U639qSebwZS3Ue2jKvbrUOBCcewcHRpsWCCQNTjvxQAiwk+4vskgLsrc2XzrgWsS7Gkxj8KlRlJi1uAdKdJQCNfEXNVU789sT+X07D9m91Qx6gvcdWh

In [15]:
accumulated.container.id

'container_011CV2TTXsJmbXMw6hXuX3ms'

In [12]:
def extract_file_ids(response):
    file_ids = []
    for item in response.content:
        if item.type == 'bash_code_execution_tool_result':
            content_item = item.content
            if content_item.type == 'bash_code_execution_result':
                for file in content_item.content:
                    if hasattr(file, 'file_id'):
                        file_ids.append(file.file_id)
    return file_ids

gen_file_ids = extract_file_ids(accumulated)
gen_file_ids


['file_011CV2TqdZjHc9gyJZ51PSGa']

In [13]:
# Download the created files
for file_id in gen_file_ids:
    file_metadata = client.beta.files.retrieve_metadata(file_id)
    file_content = client.beta.files.download(file_id)
    file_content.write_to_file(f'files/{file_metadata.filename}')
    print(f"Downloaded: {file_metadata.filename}")

Downloaded: Tata_Motors_Balance_Sheet_FY2024-25.xlsx


In [10]:
file_id = "file_011CV2R3aJGA4a655cY3DEYP"

In [ ]:
# Step 2: Use the file with streaming and thinking enabled
# Build the message content
message_content = [
    {"type": "container_upload", "file_id": file_id},
    {"type": "text", "text": "Extract the first page of the pdf as an image and return it to me. Also get the weather in San Francisco."}
]
get_weather = {
  "name": "get_weather",
  "description": "Get the current weather in a given location",
  "input_schema": {
    "type": "object",
    "properties": {
      "location": {
        "type": "string",
        "description": "The city and state, e.g. San Francisco, CA"
      },
      "unit": {
        "type": "string",
        "enum": ["celsius", "fahrenheit"],
        "description": "The unit of temperature, either 'celsius' or 'fahrenheit'"
      }
    },
    "required": ["location"]
  }
}
with client.beta.messages.stream(
    model="claude-sonnet-4-5",
    thinking={
        "type": "enabled",
        "budget_tokens": 12400
    },
    max_tokens=64000,
    betas=["files-api-2025-04-14", "code-execution-2025-08-25", "web-fetch-2025-09-10"],
    messages=[
        {
            "role": "user",
            "content": message_content
        }
    ],
    tools=[{
        "type": "code_execution_20250825",
        "name": "code_execution"
    },{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 10
    },{
        "type": "web_fetch_20250910",
        "name": "web_fetch",
        "max_uses": 10
    },
    get_weather
    ],
    system="Solve user's problem step by step.",
    # mcp_servers=[
    #     {
    #         "type": "url",
    #         "url": "https://mcp.exa.ai/mcp",
    #         # "headers": {},
    #         "name": "exa-mcp-server",
    #         "tool_configuration": {
    #             "enabled": True,
    #             "allowed_tools": ["web_search_exa", "crawling_exa", "company_research_exa", "linkedin_search_exa"]
    #         },
    #         "authorization_token": "d9ec1138-57c0-4210-b4ef-05e3e508d5d1"
    #     }
    # ]
) as stream1:
    # Stream the text output
    # for text in stream1.text_stream:
    #     print(text, end="", flush=True)
    # print()
    for event in stream1:
        if event.type == "message_start":
            # Message Start Event - can contain message id, type, input tokens, etc.
            # Exclusive type
            pass
        elif event.type == "content_block_start":
            # Can contain content block type (text, server_tool_use, tool_use, thinking, web_search_tool_result)
            # event.content_block.type, text is usually empty.
            # event.content_block.type
            # event.content_block.content
            pass
        elif event.type =="content_block_delta":
            pass
        elif event.type == "content_block_stop":
            if event.content_block.type == "tool_use":
                event.content_block.input
                pass
            pass
        elif event.type == "message_delta":
            pass
        elif event.type == "message_stop":
            pass
        
        if event.type == "text":
            # event.text 
            # event.snapshot
            print(event.text, sep="", end="", flush=True)
            pass
        elif event.type == "thinking":
            event.thinking
            event.snapshot
            pass
        elif event.type == "input_json":
            event.partial_json
            event.snapshot
            pass
        elif event.type == "citation":
            event.citation
            event.snapshot
            pass
        elif event.type == "signature":
            event.signature
            pass
            
# Get the accumulated final message (available after context manager exits)
accumulated1 = stream1.get_final_message()
snap1 = stream1.current_message_snapshot

I'll help you extract the first page of the PDF as an image and get the weather in San Francisco. Let me start by doing both tasks.

In [41]:
print(accumulated1.to_json())

{
  "id": "msg_012TXNQSG2Z15e2K65pCUo1B",
  "container": null,
  "content": [
    {
      "signature": "EtAICkYICRgCKkAd5Aov8Scttzk4lnSZY5lCvD/85MOsix2FybI1u18twNImg0W1qrs/70wBrg2GYVWsZH55PFo5sayay1YHw+G0EgyRav3KiMOCd9EZaygaDJnIhzlcvzl98L7tDSIwgHWNF/D7L2b6qrziG2YhNnisDLAMyxL1XaXoIlBZmWSwl5gh388jAhKWbS+4UwlRKrcHInETHBfZO5UC0Yhej1fuuLWwDjQf1N36qn9udzWbKzepME+1wcgSpU3otfle++WGR9VCDmGhMfJutrmbY7e59drt1O12knIZDxiz2Jfuj6b9hSyOMPWgeVGoCOyt0XN7zkTTYeG9h4YYEBT/cmwSnswsG3Cld5MveJSEDbMsT4zL/anW8m4h0acwR4TklMuv1BGkbSsdKtmBb3UHt+MK1c93QD7scR5RjmRqQSX9xTN7is7j7AnVf3cSdYGA6FMZbqDZtcmMHG00qCygXJkemz3QVG+UOr4PiQKeUVbI9NPxURMJ44pG/+W5O3l95E7RA1rNgQOugNcOD+wWWuGT2RJ+Ax9Qy2nLywGAOQJXmCo7VNTOCYub4js6GzVpl+kHPW94ufDg6XIFIB9al1D3qZ/8+oqWgmT7hBagl8tpkeKEZrAEHR2CcsiDfXfCk8032I9HrubS25bkgKQiWfIALvQUr/YJ6NxoHAflRJsvcvkVYvrc3qX/eFFvIWvizQfauelsT49WtZXmZlCwhl0dJppkzJ1B6P6MO4Cu30uLYGdgGJM74779xTxqAAbD3hL0lfZI5b1hRYy3JJI5Mo6WNabXD1wFt3wa0btkabmaky2XflxRL/TM9wEguWC/ug/igJL8RJ/pRqli7DSY1UbxX2gKCdoz4dguXjVcGh1M2MfT14dC

In [30]:
print(json.dumps(accumulated1.to_dict()['content'], indent=2, default=str))

[
  {
    "signature": "EooHCkYICRgCKkCcypCScGSAKLgunqtGoAplIDs6ttoV3EaU8I95NNEP/ZL546dIkR780RkAXpSRPZAdTpivlG/EiECVpnP2y7SEEgznf761/mkzw3Z5Yk8aDNfZIBiIIpf9IFWiUiIwTJLqTjUAAXMOO5/JhrrpyBYZ5RHK9ZWtcgr6+Rq7Z9O9BwCIas/IrXJqUXcyUwJ6KvEFTrJN7q22MQ1lLJGVbWuNIE0ambf2HknKhfUArqSYZgPzEAQ3nwe9LUo+/7ikfFUEb/V+/PshN0k65jRFQtwAlNoyciegnrHzrsXJpUvHbWQ1HEfc7zHBXxbANC3PbX4uSFtqA9ivicywdSwxJ7vP2xZbvCV5/wTYQ603FEvp1AO+ndoLvSGhS6lY+TgXSkdsqTfg3OMmFUmca1rglzPPHIIRF3vKGkXvkEZ3fYIGYfB3NKoor5e5G9+/UqaFcCmlVwAIs1amCuKm427VlRkUvBH5lYEZWAW+ZOT54w9vhoMQDectD5JjXxnbKLb0JkjpY34+fMLKNiMie/Frj0QUOmvweU/yFLclwOHFi7/UYzqc8fO7WipAhmBeBR3AcWLpYhUWAOogqkEMeZq8ffP0ghnPH0d6zFcLMQtpkQ0W4LPdCDj45jJbd1wrLXtdJh1cu4VR/ShHz7vg8ADpm26beEwlYk2fg6K3sqWKXUDQKm7siKxnHFXWiHdnsmTeU/kIHpexfsAAVvIEBWQJy/qK6bQBRIbkPgYUqa9bg8odfRH3sakfY1J3gsry6UElJInGNKa5dq0FqohNSyVl7oufTcahxeP+cQ2ym42jEPZ/QpH3r0HOA7skMvjeFmPRS0fpcVnRxGJ7emQQ+xoDxfgwrJactg6f/+8aa/Jgn9J0k4REwjdhaI0+EyaVJKauhfNlGaGKkBW5ndh4d8rcFiOXHNX154MDPZuww6mldoarjeDGvyGqr/OuWNT5IuTnQITC

In [44]:
# Step 2: Use the file with streaming and thinking enabled
# Build the message content
new_messages = [{
            "role": "user",
            "content": message_content
        }]

new_messages.append(
    {
        "role": accumulated1.role,
        "content": accumulated1.content
    }
)

new_messages.append(
    {
        "role": "user",
        "content": [
            {
                "type": "tool_result",
                "tool_use_id": "toolu_01DX8ob46Urascd1XeumedFu",
                "content": "Here is the weather in San Francisco: 60 degrees and cloudy"
            }
        ]
    }
)
# new_messages.append(
#     {
#         "role": "user",
#         "content": "What did you do in previous step?"
#     }
# )

with client.beta.messages.stream(
    model="claude-sonnet-4-5",
    # container=accumulated1.container.id,
    thinking={
        "type": "enabled",
        "budget_tokens": 12400
    },
    max_tokens=64000,
    betas=["files-api-2025-04-14", "code-execution-2025-08-25", "web-fetch-2025-09-10"],
    messages=new_messages,
    tools=[{
        "type": "code_execution_20250825",
        "name": "code_execution"
    },{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 10
    },{
        "type": "web_fetch_20250910",
        "name": "web_fetch",
        "max_uses": 10
    },
    get_weather
    ],
) as stream2:
    # Stream the text output
    for text in stream2.text_stream:
        print(text, end="", flush=True)
    print()

# Get the accumulated final message (available after context manager exits)
accumulated2 = stream2.get_final_message()
snap2 = stream2.current_message_snapshot

Great! Now let me create a Python script to extract the first page of the PDF as an image:Perfect! I've completed both tasks:

## Weather in San Francisco
The current weather in San Francisco is **60°F and cloudy**.

## PDF First Page
I've successfully extracted the first page from the "tata-motor-IAR-2024-25.pdf" file and saved it as an image (PNG format at 200 DPI). The image file "first_page.png" has been exported to you.


In [45]:
print(accumulated2.to_json())

{
  "id": "msg_01BsWBi3V8B5T32GsaRXyC3F",
  "container": {
    "id": "container_011CV514LdicQPXowaEGBoxe",
    "expires_at": "2025-11-13T02:14:24.601879Z"
  },
  "content": [
    {
      "content": {
        "content": [],
        "return_code": 2,
        "stderr": "ls: cannot access '/files/input/b326382d8e25c9ab': No such file or directory\n",
        "stdout": "",
        "type": "bash_code_execution_result"
      },
      "tool_use_id": "srvtoolu_01FLXuhwhGZkaWSvHy4mt7TA",
      "type": "bash_code_execution_tool_result"
    },
    {
      "id": "srvtoolu_01TJ29LXZksHvszggoV7kMfg",
      "input": {
        "command": "find /files -name \"*.pdf\" 2>/dev/null | head -20"
      },
      "name": "bash_code_execution",
      "type": "server_tool_use"
    },
    {
      "content": {
        "content": [],
        "return_code": 0,
        "stderr": "",
        "stdout": "/files/input/b326382d8e25c9ab/tata-motor-IAR-2024-25.pdf\n",
        "type": "bash_code_execution_result"
      },
   

In [32]:
print(accumulated2.to_json())

{
  "id": "msg_01Axy6pCLii8HJ286AWF19bi",
  "container": null,
  "content": [
    {
      "signature": "EpoECkYICRgCKkCG8gcvDSEpEc/oqapFBdSsSzKFgqT9lNpGNB5knMKVjRPW9H84bKprKObLfVQw5h+P2CKbxOIpi7ITaCh0YBWyEgxXFpi/ayn/AxtnlXUaDKXbdrxKTTwD0iK/PyIwu9hx93I8GK33O6xCdcVHziS6OL8AbsUDve6BLM0Oa15uZn7BBb+2qY+zQGO5UK/uKoED783MZssOK2zk7Ke5Thj6sAXNRwfVBCZNeTIRyILM4JB3+Z++1pC4o+cTdo0r+XG28KT2NSrjRtrRrSGJMxcIV0Bi9vZoA18j8yVHCsEwkX/Z2QPdjHVK5kl1yAWWFQd7+6mD+lU1H2gqwRjqM/aV7tJli8wXzio6kzXMnITbKCxBL/XvzxVT0uvYxJ1k/GTOxlhQ7/1fX/uOV5Buz6uV44KX2ueZAtS34O8ZpJZSwJFfBu67Pt/nte5g/wNcY3YtsxUNACS5VdqZCbDMLTU446jaQJfWZ11s7c0SDxW4zZxRRbgVVwJp0tc5GGc31wmxlrhqoTGsgHGYhvUb6WQS2RkPFg81McVg/wziybuh4pKMp+/IuMQRcNm9jXSU9tVZHRTkTqhqb7TRaJFJ6jGVH+yBASme/rLpQGUquiMHjXrjnB7cuCOWMKKZ9SQ4dCnJ9hDvPtoCZd8AsvrLYmJ3rc871+UOREz+3EWPEpeL+WRJ15rhr+Z5TPqHRNbBA5sfKBgB",
      "thinking": "The user is asking me to explain what I did in the previous step. Let me break down the process clearly.\n\nIn the previous step, I:\n1. Created a Pyt

## LiteAgent - Call

In [33]:
import asyncio
import litellm
from lite_agent.streaming import render_stream

async def stream_litellm():
    # Build message content
    file_id = "file_011CV2R3aJGA4a655cY3DEYP"
    message_content = [
        # {"type": "container_upload", "file_id": file_id},
        {"type": "text", "text": "Extract the first page of this pdf and return as an image to me."}
    ]
    
    # Create queue for streaming output
    queue = asyncio.Queue()
    
    # Start litellm completion with streaming
    response = await litellm.acompletion(
        model="anthropic/claude-sonnet-4-5",
        custom_llm_provider="anthropic",
        messages=[{"role": "user", "content": message_content}],
        max_tokens=64000,
        thinking={"type": "enabled", "budget_tokens": 12400},
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        betas=["files-api-2025-04-14", "code-execution-2025-08-25"],
        stream=True,
        stream_options={"include_usage": True}
    )
    
    # Render stream with lite_agent's formatter
    render_task = asyncio.create_task(render_stream(response, queue))
    
    # Consume and print from queue
    while True:
        try:
            chunk = await asyncio.wait_for(queue.get(), timeout=0.1)
            print(chunk, end="", flush=True)
        except asyncio.TimeoutError:
            if render_task.done():
                while not queue.empty():
                    print(queue.get_nowait(), end="", flush=True)
                break
    
    # Get final accumulated message
    accumulated = await render_task
    return accumulated

# Run it
lres = await stream_litellm()


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



BadRequestError: litellm.BadRequestError: AnthropicException - b'{"type":"error","error":{"type":"invalid_request_error","message":"tools.0: Input tag \'code_execution_20250825\' found using \'type\' does not match any of the expected tags: \'bash_20250124\', \'custom\', \'text_editor_20250124\', \'text_editor_20250429\', \'text_editor_20250728\', \'web_search_20250305\'"},"request_id":"req_011CV4bCNiAN2YJ9pbhhztoi"}'

In [ ]:
litellm.AnthropicConfig